In [3]:
import pandas as pd
pd.set_option('display.max_columns',15)
list_of_refugee = pd.read_csv('RefugeeList.csv')
print(list_of_refugee)

  Family ID Lead Family Member Name Lead Family Member Surname Camp ID  \
0       1A1                    Mary                    Collins      A1   
1       1A2                    Khan                     Samuel      A2   
2       2A1                    Mark                      Brown      A1   
3       3A1                 Michael                      Smith      A1   
4       2A2                     Sam                    Roberts      A2   
5       3A2                  Sophie                      Jones      A2   

  Mental State Physical State  No. Of Family Members  
0         good            bad                      3  
1         good            bad                      4  
2          bad            bad                      2  
3         good           good                      3  
4          bad           good                      5  
5          bad            bad                      2  


In [4]:
list_of_refugee['Family ID']

0    1A1
1    1A2
2    2A1
3    3A1
4    2A2
5    3A2
Name: Family ID, dtype: object

In [39]:
data = pd.DataFrame([['69','Bob','Nightmare','Kaluga','good','bad','many']], columns = list_of_refugee.columns )
list_of_refugee.append(data)
#data

C:\Users\fsuli\AppData\Local\Temp\ipykernel_18220\2108619073.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  list_of_refugee.append(data)


,Family ID,Lead Family Member Name,Lead Family Member Surname,Camp ID,Mental State,Physical State,No. Of Family Members
0,1A1,Mary,Collins,A1,good,bad,3
1,1A2,Khan,Samuel,A2,good,bad,4
2,2A1,Mark,Brown,A1,bad,bad,2
3,3A1,Michael,Smith,A1,good,good,3
4,2A2,Sam,Roberts,A2,bad,good,5
5,3A2,Sophie,Jones,A2,bad,bad,2
0,69,Bob,Nightmare,Kaluga,good,bad,many


In [104]:
def amend_refugee_profile():
    '''familyID, name, surname, campID, mentalState, physicalState, no_of_relatives'''
    print('AMEND REFUGEE PROFILE')

    while True:
        iD = input('Please choose Family ID you would like to ammend: ')
        if iD not in list(list_of_refugee['Family ID']):
            print('Please enter valid Family ID')
            continue
        print(f'\nPlease Choose which values you would like to ammend for Family {iD}')
        print('-'*25)
        print('Input indices corresponding to value you wish to ammend separated by commas ","')
        print('eg: "1,2" for amending "Lead Family Member Name" and "Lead Family Member Surname"')
        print('[1] - "Lead Family Member Name"')
        print('[2] - "Lead Family Member Surname"')
        print('[3] - "Camp ID"')
        print('[4] - "Mental State"')
        print('[5] - "Physical State"')
        print('[6] - "No. Of Family Members"\n')

        while True:
            ops = input('Indices: ')
            try:
                ops = [int(i) for i in ops.split(',')]
            except:
                print('Please input valid indices')
                continue
            check = [not (i<1 or i>6) for i in ops]
            if any(i is False for i in check):
                print('Please input valid indices')
                continue
            break
        
        for i in ops:
            change = input(f'Please select new value for {list_of_refugee.columns[i]}')
            list_of_refugee.at[list_of_refugee.index[list_of_refugee['Family ID'] == iD][0],list_of_refugee.columns[i]]=change
        
        break
    
amend_refugee_profile()   

AMEND REFUGEE PROFILE

Please Choose which values you would like to ammend
-------------------------
Input indices corresponding to value you wish to ammend separated by commas ","
eg: "1,2" for amending "Lead Family Member Name" and "Lead Family Member Surname"
[1] - "Lead Family Member Name"
[2] - "Lead Family Member Surname"
[3] - "Camp ID"
[4] - "Mental State"
[5] - "Physical State"
[6] - "No. Of Family Members"
[123] [False]
Please input valid indices
Please input valid indices!
[1, 90] [True, False]
Please input valid indices
[1, 2, 3] [True, True, True]


In [105]:
list_of_refugee

,Family ID,Lead Family Member Name,Lead Family Member Surname,Camp ID,Mental State,Physical State,No. Of Family Members
0,1A1,Fedor,Collins,A1,good,bad,3
1,1A2,Khan,Samuel,A2,good,bad,4
2,2A1,Mark,Brown,A1,bad,bad,2
3,3A1,Michael,Smith,A1,good,good,3
4,2A2,Sam,Roberts,A2,bad,good,5
5,3A2,Sophie,Jones,A2,bad,bad,2


In [86]:
a = 'sad'
try:
    ops = [int(i) for i in a.split(',')]
    #ops = [i for i in a if len(i)!=0]
    for i in ops:
        if i<1 or i>6:
            print('Please input valid indices') 
except:
    print('Please input valid indices')

Please input valid indices


In [80]:
a

'1,12,'

In [53]:
#df.at[df.index[df['Family ID'] == '1A1'][0],'Lead Family Member Surname'] = 'S'
df['Lead Family Member Surname']

0    Sulitskiy
1       Samuel
2        Brown
3        Smith
4      Roberts
5        Jones
Name: Lead Family Member Surname, dtype: object

In [50]:
df

,Family ID,Lead Family Member Name,Lead Family Member Surname,Camp ID,Mental State,Physical State,No. Of Family Members
0,1A1,Billy,Collins,A69,good,bad,3
1,1A2,Khan,Samuel,A2,good,bad,4
2,2A1,Mark,Brown,A1,bad,bad,2
3,3A1,Michael,Smith,A1,good,good,3
4,2A2,Sam,Roberts,A2,bad,good,5
5,3A2,Sophie,Jones,A2,bad,bad,2


In [39]:
df.at[df.index[df['Family ID'] == '1A1'][0],'Lead Family Member Name']='Billy'

In [51]:
df

,Family ID,Lead Family Member Name,Lead Family Member Surname,Camp ID,Mental State,Physical State,No. Of Family Members
0,1A1,Billy,Collins,A69,good,bad,3
1,1A2,Khan,Samuel,A2,good,bad,4
2,2A1,Mark,Brown,A1,bad,bad,2
3,3A1,Michael,Smith,A1,good,good,3
4,2A2,Sam,Roberts,A2,bad,good,5
5,3A2,Sophie,Jones,A2,bad,bad,2


In [26]:
df[df.columns[0]]

0    1A1
1    1A2
2    2A1
3    3A1
4    2A2
5    3A2
Name: Family ID, dtype: object

In [108]:
df = pd.read_csv('VolounteersData.csv').set_index('Username')
#df['First name '] = df['First name '].astype(str)
vol_dict = df.to_dict(orient='index')
df

,First name,Second name,Phone,Camp ID,Avability,Status
Username,,,,,,
Volunteer1,Mike,Smith,441234567,A1,15h,1
Volunteer2,Jane,Smith,449876543,A1,40h,1
Volunteer3,Meghan,Brown,441312456,A2,10h,1
Volunteer4,Sam,Richards,446745891,A1,30h,1
Volunteer5,Amber,Robinson,448967124,A2,17h,1
Volunteer6,Alex,Gold,445689541,A2,8h,1
Volunteer7,Henry,Vincent,445897352,A1,33h,1
Volunteer8,Jessica,Vincent,442356789,A1,18h,1
Volunteer9,George,Thompson,445612345,A1,10h,1


In [121]:
a = vol_dict['Volunteer1']['Camp ID']
a

'A1'

In [120]:
a['First name ']

'Mike'